In [ ]:
# Required libraries
import os
import cv2
import math
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from matplotlib import cm
import matplotlib.colors as mcolors
from sklearn.cluster import KMeans
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.ndimage import label, find_objects, binary_erosion, binary_dilation, distance_transform_edt
from scipy import stats as scipy_stats
from scipy.signal import find_peaks, savgol_filter
from sklearn.metrics import mean_squared_error
from scipy.spatial import Delaunay, distance
from sklearn.cluster import DBSCAN
from skimage import filters
from skimage.filters import threshold_otsu, threshold_multiotsu
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import networkx as nx

In [ ]:
# Define folder structure
input_dir = 'input_data'
output_dir = 'output_data'

# Input staleite data, i.e.: https://browser.dataspace.copernicus.eu/
optical_image_path = os.path.join(input_dir, 'Gdansk_2025-04-10-00_00_2025-04-10-23_59_Sentinel-2_Quarterly_Mosaics_True_Color_Cloudless.jpg')
sar_image_path = os.path.join(input_dir, 'Gdansk_2025-04-07-00_00_2025-04-07-23_59_Sentinel-1_IW_VV+VH_VV_-_decibel_gamma0.jpg')

# Extract base name for input files (without extension)
base_name = os.path.splitext(os.path.basename(optical_image_path))[0].split('_optical')[0]

# Define resolution: i.e. 20m per pixel
satellite_pixel_resolution = 15

# Set ticks intervals for different plots
image_x_tick_interval = 5000  # meters (5km)
image_y_tick_interval = image_x_tick_interval  # meters (5km)
gradient_x_tick_interval = 1000  # meters (1km)

In [ ]:
# Decompose color histogram tool
def decompose_histogram(image, num_peaks=3, value_range=(0, 2), num_bins=50, exclude_zeros=False):
    """
    Decompose an image histogram into a specified number of Gaussian components.
    
    Parameters:
    -----------
    image : 2D numpy array
        The input image to be analyzed
    num_peaks : int
        Number of Gaussian peaks to fit (default: 3)
    value_range : tuple
        Range of values to consider for the histogram (default: (0, 2))
    num_bins : int
        Number of bins for the histogram (default: 50)
    exclude_zeros : bool
        Whether to exclude zero values from the analysis (default: False)
        
    Returns:
    --------
    dict
        Dictionary containing the fitted parameters and component data
    """
    
    # Flatten the image data
    flat_data = image.flatten()
    
    # Exclude zeros if requested
    if exclude_zeros:
        flat_data = flat_data[flat_data != 0]
        
        # Check if we have any data left after excluding zeros
        if len(flat_data) == 0:
            print("Warning: No non-zero data points found. Cannot perform analysis.")
            return None
            
        print(f"Excluded zeros. Analyzing {len(flat_data)} non-zero values.")
    
    # Create histogram data
    hist, bin_edges = np.histogram(flat_data, bins=num_bins, range=value_range)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    
    # Define a function for multiple Gaussian components
    def multi_gaussian(x, *params):
        """
        Sum of multiple Gaussian distributions.
        params format: [a1, mu1, sigma1, a2, mu2, sigma2, ...]
        """
        y = np.zeros_like(x)
        for i in range(0, len(params), 3):
            a = params[i]
            mu = params[i+1]
            sigma = params[i+2]
            y += a * np.exp(-(x - mu)**2 / (2 * sigma**2))
        return y
    
    # Define a single Gaussian function for intersection analysis
    def single_gaussian(x, a, mu, sigma):
        """Single Gaussian distribution."""
        return a * np.exp(-(x - mu)**2 / (2 * sigma**2))
    
    # Create initial guess and bounds for parameters
    initial_guess = []
    lower_bounds = []
    upper_bounds = []
    
    max_amplitude = np.max(hist)
    value_min, value_max = value_range
    range_width = value_max - value_min
    
    # Get peak indices from histogram to improve initial guesses
    # Find local maxima in the histogram
    from scipy.signal import find_peaks
    peak_indices, _ = find_peaks(hist, height=max(hist) * 0.2)
    
    # If we found some peaks, use them for initial guesses
    if len(peak_indices) > 0 and len(peak_indices) <= num_peaks:
        peak_positions = bin_centers[peak_indices]
        peak_heights = hist[peak_indices]
        
        # Sort by height, descending
        sorted_indices = np.argsort(peak_heights)[::-1]
        peak_positions = peak_positions[sorted_indices]
        peak_heights = peak_heights[sorted_indices]
        
        # Use detected peaks first, then add evenly spaced ones if needed
        for i in range(num_peaks):
            if i < len(peak_positions):
                # Use detected peak
                amplitude = peak_heights[i]
                mean = peak_positions[i]
            else:
                # Add evenly spaced peaks for remaining
                amplitude = max_amplitude / (num_peaks * 2)
                mean = value_min + range_width * (i + 0.5) / num_peaks
            
            # Amplitude
            initial_guess.append(amplitude)
            lower_bounds.append(0)
            upper_bounds.append(np.inf)
            
            # Mean
            initial_guess.append(mean)
            lower_bounds.append(value_min)
            upper_bounds.append(value_max)
            
            # Standard deviation - wider initial guess for better convergence
            initial_guess.append(range_width / (4 * num_peaks))
            lower_bounds.append(range_width / (50 * num_peaks))  # Allow narrower peaks
            upper_bounds.append(range_width)  # Allow wider peaks
    else:
        # Fall back to evenly spaced means if peak detection fails
        for i in range(num_peaks):
            # Amplitude
            initial_guess.append(max_amplitude / num_peaks)
            lower_bounds.append(0)
            upper_bounds.append(np.inf)
            
            # Mean (evenly distribute initial guesses across the range)
            mean_guess = value_min + range_width * (i + 0.5) / num_peaks
            initial_guess.append(mean_guess)
            lower_bounds.append(value_min)
            upper_bounds.append(value_max)
            
            # Standard deviation - wider initial guess for better convergence
            initial_guess.append(range_width / (4 * num_peaks))
            lower_bounds.append(range_width / (50 * num_peaks))  # Allow narrower peaks
            upper_bounds.append(range_width)  # Allow wider peaks
    
    # Perform the curve fit with increased max iterations and tweaked optimization parameters
    try:
        params, covariance = curve_fit(
            multi_gaussian, 
            bin_centers, 
            hist, 
            p0=initial_guess,
            bounds=(lower_bounds, upper_bounds),
            maxfev=10000,  # Increase maximum function evaluations
            method='trf',  # Use Trust Region Reflective algorithm
            ftol=1e-6,     # Function tolerance for termination
            xtol=1e-6      # Parameter tolerance for termination
        )
    except RuntimeError as e:
        print(f"Fitting error: {e}")
        print("Trying alternative optimization approach...")
        
        # Try a simpler approach: reduce constraints and use different method
        try:
            # Remove bounds and use different method
            params, covariance = curve_fit(
                multi_gaussian, 
                bin_centers, 
                hist, 
                p0=initial_guess,
                maxfev=20000,      # More iterations
                method='lm'        # Levenberg-Marquardt algorithm (no bounds but can be more robust)
            )
            print("Alternative fitting approach succeeded!")
        except RuntimeError as e2:
            print(f"Alternative fitting also failed: {e2}")
            return None
    
    # Calculate the fitted values
    fitted_y = multi_gaussian(bin_centers, *params)
    
    # Calculate individual Gaussian components
    components = []
    for i in range(0, len(params), 3):
        a = params[i]
        mu = params[i+1]
        sigma = params[i+2]
        component = a * np.exp(-(bin_centers - mu)**2 / (2 * sigma**2))
        components.append(component)
    
    # Calculate the contribution of each component
    total_area = np.sum(fitted_y)
    contributions = [np.sum(comp) / total_area * 100 for comp in components]
    
    # Calculate intersections between adjacent Gaussian components
    intersections = []
    
    # Function to find intersection point between two Gaussian curves
    def find_intersection(params1, params2, x_guess):
        a1, mu1, sigma1 = params1
        a2, mu2, sigma2 = params2
        
        def func(x):
            return single_gaussian(x, a1, mu1, sigma1) - single_gaussian(x, a2, mu2, sigma2)
        
        try:
            result = fsolve(func, x_guess)
            # Check if the solution is within our range
            if value_min <= result[0] <= value_max:
                # Verify it's actually an intersection
                y1 = single_gaussian(result[0], a1, mu1, sigma1)
                y2 = single_gaussian(result[0], a2, mu2, sigma2)
                if np.isclose(y1, y2, rtol=1e-3) and y1 > 0:
                    return result[0], y1
        except:
            pass
        
        return None, None
    
    # Sort the components by their means
    component_indices = list(range(num_peaks))
    component_indices.sort(key=lambda i: params[i*3+1])
    
    intersection_data = []
    
    # Find intersections between adjacent components
    for i in range(len(component_indices)-1):
        idx1 = component_indices[i]
        idx2 = component_indices[i+1]
        
        # Extract parameters for each component
        params1 = params[idx1*3:idx1*3+3]
        params2 = params[idx2*3:idx2*3+3]
        
        # Initial guess for intersection: midpoint between means
        x_guess = (params1[1] + params2[1]) / 2
        
        # Find intersection
        x_intersect, y_intersect = find_intersection(params1, params2, x_guess)
        
        if x_intersect is not None:
            intersections.append((x_intersect, y_intersect))
            intersection_data.append({
                'components': (idx1+1, idx2+1),
                'x_value': x_intersect,
                'y_value': y_intersect,
                'relative_height': y_intersect / max(np.max(components[idx1]), np.max(components[idx2]))
            })
    
    # Create a summary plot
    plt.figure(figsize=(10, 6))
    
    # Plot original histogram envelope
    plt.plot(bin_centers, hist, 'ko', label='Original Data', alpha=0.5)
    
    # Plot the combined fit
    plt.plot(bin_centers, fitted_y, 'r-', linewidth=2, label='Combined Fit')
    
    # Plot individual Gaussian components
    colors = ['g', 'b', 'm', 'c', 'y', 'orange', 'purple', 'brown', 'pink', 'gray']
    component_params = []
    
    for i in range(num_peaks):
        idx = i * 3
        a = params[idx]
        mu = params[idx+1]
        sigma = params[idx+2]
        contrib = contributions[i]
        
        color = colors[i % len(colors)]
        plt.plot(bin_centers, components[i], f'{color}--', linewidth=1.5, 
                 label=f'Gaussian {i+1}: μ={mu:.2f}, σ={sigma:.2f}, {contrib:.1f}%')
        
        component_params.append({
            'amplitude': a,
            'mean': mu,
            'std_dev': sigma,
            'contribution': contrib
        })
    
    # Mark intersection points
    for x, y in intersections:
        plt.plot(x, y, 'ro', markersize=8)
        plt.axvline(x=x, color='k', linestyle=':', alpha=0.5)
    
    # Add labels and title
    plt.xlabel('Pixel Value')
    plt.ylabel('Frequency')
    if exclude_zeros:
        plt.title(f'Decomposition of Histogram into {num_peaks} Gaussian Components (Zeros Excluded)')
    else:
        plt.title(f'Decomposition of Histogram into {num_peaks} Gaussian Components')
    plt.legend()
    plt.grid(alpha=0.3)
    
    # Display the plot
    plt.tight_layout()
    plt.show()
    
    # Print the fitted parameters
    print(f"Fitted Parameters for {num_peaks} Gaussian Components:")
    for i, comp in enumerate(component_params):
        print(f"Gaussian {i+1}: Amplitude={comp['amplitude']:.1f}, "
              f"Mean={comp['mean']:.3f}, StdDev={comp['std_dev']:.3f}, "
              f"Contribution={comp['contribution']:.1f}%")
    
    # Print intersection information
    if intersection_data:
        print("\nIntersection Points Between Gaussian Components:")
        for i, idata in enumerate(intersection_data):
            print(f"Intersection {i+1}: Between Gaussian {idata['components'][0]} and {idata['components'][1]}")
            print(f"  Position: x={idata['x_value']:.3f}, y={idata['y_value']:.1f}")
            print(f"  Relative height: {idata['relative_height']*100:.1f}% of max component height")
    else:
        print("\nNo valid intersections found between Gaussian components.")
    
    # Return results
    return {
        'bin_centers': bin_centers,
        'histogram': hist,
        'fitted_curve': fitted_y,
        'components': components,
        'component_params': component_params,
        'raw_params': params,
        'intersections': intersection_data
    }

In [ ]:
# Load and process optical image
optical_image = cv2.imread(optical_image_path)
optical_image_rgb = cv2.cvtColor(optical_image, cv2.COLOR_BGR2RGB)
optical_image_gray = cv2.cvtColor(optical_image, cv2.COLOR_BGR2GRAY)  # Simplified conversion

# Edge detection with Sobel
optical_image_sobelx = cv2.Sobel(optical_image_gray, cv2.CV_64F, 1, 0, ksize=3)
optical_image_sobely = cv2.Sobel(optical_image_gray, cv2.CV_64F, 0, 1, ksize=3)
optical_image_sobel_combined = cv2.magnitude(optical_image_sobelx, optical_image_sobely)
optical_image_sobel_combined = cv2.normalize(optical_image_sobel_combined, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

# Create edge density map
kernel_size = 15
optical_image_density_map = cv2.GaussianBlur(optical_image_sobel_combined, (kernel_size, kernel_size), 0)
# Fix: Convert to float before normalizing
optical_image_density_map = optical_image_density_map.astype(float) / np.max(optical_image_density_map)

# Load and normalize SAR image
sar_image = cv2.imread(sar_image_path, cv2.IMREAD_GRAYSCALE).astype(float) / 255.0

# Combine images and apply Non-Local Means smoothing
combined_image = optical_image_density_map + sar_image
combined_image_uint8 = np.clip(combined_image * 127.5, 0, 255).astype(np.uint8)
smoothed_image_uint8 = cv2.fastNlMeansDenoising(combined_image_uint8, None, 10, 7, 21)
combined_image = smoothed_image_uint8.astype(float) / 127.5

In [ ]:
# Define water_threshold and urban_threshold based of histograms decomposition
result = decompose_histogram(combined_image, num_peaks=3, exclude_zeros=False)
intersection_x_values = []
if result and 'intersections' in result:
    for intersection in result['intersections']:
        intersection_x_values.append(intersection['x_value'])

# Define thresholds for segmentation
#water_threshold = intersection_x_values[0]
#urban_threshold = intersection_x_values[1]
water_threshold = 0.2
urban_threshold = 0.6

In [ ]:
# Create initial segmentation
segmentation = np.zeros_like(combined_image, dtype=np.uint8)
segmentation[(combined_image < water_threshold)] = 1  # Water
segmentation[(combined_image >= water_threshold) & (combined_image < urban_threshold)] = 2  # Terrain
segmentation[(combined_image >= urban_threshold)] = 3  # Urban

# Process urban mask
urban_mask = (segmentation == 3).astype(np.uint8) * 255
kernel = np.ones((5, 5), np.uint8)
urban_mask_closed = cv2.morphologyEx(
    cv2.dilate(urban_mask, kernel, iterations=1), 
    cv2.MORPH_CLOSE, 
    kernel
)

# Filter small urban patches
num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(urban_mask_closed, connectivity=8)
urban_mask_filtered = np.zeros_like(urban_mask_closed)
for i in range(1, num_labels):
    if stats[i, cv2.CC_STAT_AREA] >= 100:  # min_urban_area = 100
        urban_mask_filtered[labels == i] = 255

# Create final segmentation
improved_segmentation = segmentation.copy()
improved_segmentation[improved_segmentation == 3] = 2  # Reset urban to terrain
improved_segmentation[urban_mask_filtered == 255] = 3  # Set filtered urban areas

# Apply colormap for visualization
colormap = np.array([
    [0, 0, 0],      # Background
    [0, 0, 255],    # Water (blue)
    [0, 255, 0],    # Terrain (green)
    [255, 0, 0]     # Urban (red)
], dtype=np.uint8)
segmented_image = colormap[improved_segmentation]

# Create masks for density analysis
urban_mask = improved_segmentation == 3
urban_mask_numeric = urban_mask.astype(float)
urban_density = combined_image * urban_mask_numeric

In [ ]:
# Define urban_center_threshold based of histograms decomposition
result = decompose_histogram(urban_density, num_peaks=2, exclude_zeros=True)
intersection_x_values = []
if result and 'intersections' in result:
    for intersection in result['intersections']:
        intersection_x_values.append(intersection['x_value'])

# Define thresholds for urban center segmentation
urban_center_threshold = intersection_x_values[0]
urban_centers_mask = (combined_image > urban_center_threshold) & urban_mask

In [ ]:
# Function to set axes in meters
def set_axes_in_meters(ax, img_shape):
    height, width = img_shape[:2]
    # Create x and y axes in meters
    x_meters = np.arange(0, width * satellite_pixel_resolution, satellite_pixel_resolution)
    y_meters = np.arange(0, height * satellite_pixel_resolution, satellite_pixel_resolution)
    
    # Set ticks at regular intervals (5km)
    image_x_tick_interval = 5000  # meters (5km)
    image_y_tick_interval = 5000  # meters (5km)
    
    x_ticks = np.arange(0, width * satellite_pixel_resolution, image_x_tick_interval)
    y_ticks = np.arange(0, height * satellite_pixel_resolution, image_y_tick_interval)
    
    ax.set_xticks(x_ticks / satellite_pixel_resolution)
    ax.set_yticks(y_ticks / satellite_pixel_resolution)
    ax.set_xticklabels([f'{int(x/1000)}km' for x in x_ticks])
    ax.set_yticklabels([f'{int(y/1000)}km' for y in y_ticks])
    
    ax.set_xlabel('Distance (km)')
    ax.set_ylabel('Distance (km)')

# Individual Plots - saves each image as a separate file
#------------------------------------------------------------------

# 1. Original Image
plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.imshow(optical_image_rgb)
plt.title('Original Optical Image')
set_axes_in_meters(ax, optical_image_rgb.shape)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_original.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_original.png file saved')
plt.show()
plt.close()

# 2. Sobel Edge Detection
plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.imshow(optical_image_sobel_combined, cmap='gray')
plt.title('Sobel Edge Detection')
set_axes_in_meters(ax, optical_image_sobel_combined.shape)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_sobel.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_sobel.png file saved')
plt.show()
plt.close()

# 3. Edge Density Map
plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.imshow(optical_image_density_map, cmap='gray')
plt.title('Edge Density Map (0-1 range)')
set_axes_in_meters(ax, optical_image_density_map.shape)
# Add colorbar with fixed size
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(ax.images[0], cax=cax)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_density.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_density.png file saved')
#plt.show()
plt.close()

# 4. SAR Image
plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.imshow(sar_image, cmap='gray')
plt.title('SAR Image (0-1 range)')
set_axes_in_meters(ax, sar_image.shape)
# Add colorbar with fixed size
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(ax.images[0], cax=cax)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_sar.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_sar.png file saved')
plt.show()
plt.close()

# 5. Combined Result
plt.figure(figsize=(10, 8))
ax = plt.gca()
im = plt.imshow(combined_image, cmap='viridis')
plt.title('Combined Result (0-2 range)')
set_axes_in_meters(ax, combined_image.shape)
# Add colorbar with fixed size
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_combined.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_combined.png file saved')
plt.show()
plt.close()

# 6. Segmented Image
plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.imshow(segmented_image)  # No cmap needed - it's already an RGB image
plt.title('Improved Segmentation')
set_axes_in_meters(ax, segmented_image.shape)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_segmented.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_segmented.png file saved')
plt.show()
plt.close()

# 7. Urban Density Map
plt.figure(figsize=(10, 8))
ax = plt.gca()
im = plt.imshow(urban_density, cmap='viridis')
plt.title('Urban Density Map (0-2 range)')
set_axes_in_meters(ax, urban_density.shape)
# Add colorbar with fixed size
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
plt.colorbar(im, cax=cax)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_urban_density.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_urban_density.png file saved')
#plt.show()
plt.close()

# 8. Urban Centers
plt.figure(figsize=(10, 8))
ax = plt.gca()
plt.imshow(urban_mask, cmap='gray', alpha=0.5)
plt.imshow(urban_centers_mask, cmap='Reds', alpha=0.8)
plt.title(f"Urban Centers (Density > {urban_center_threshold})")
set_axes_in_meters(ax, urban_centers_mask.shape)
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_urban_centers.png', dpi=300, bbox_inches='tight')
print(f'{output_dir}/{base_name}_urban_centers.png file saved')
plt.show()
plt.close()

In [ ]:
# Set gradient_intercept to the minimum of the urban density
gradient_intercept = urban_density.min()

# Calculate density gradients from centers
def calculate_density_gradients(centers_mask, urban_mask, density_map):
    """Calculate how density changes with distance from urban centers"""
    # Calculate distance from each urban pixel to the nearest center
    # First invert the centers mask to get distance transform
    centers_dist_transform = distance_transform_edt(~centers_mask)
    
    # Only consider distances within urban areas
    urban_distances = centers_dist_transform[urban_mask]
    urban_densities = density_map[urban_mask]
    
    # Bin distances to create gradient profile
    max_dist = int(np.max(urban_distances)) + 1
    distance_bins = range(max_dist)
    density_by_distance = []
    
    for d in distance_bins:
        # Get densities at this distance
        mask = (urban_distances >= d) & (urban_distances < d+1)
        if np.sum(mask) > 0:
            avg_density = np.mean(urban_densities[mask])
            density_by_distance.append((d, avg_density))
    
    return density_by_distance

# Calculate density gradients
density_gradients = calculate_density_gradients(urban_centers_mask, urban_mask, combined_image)

# Density gradient plot data
distances = [d for d, _ in density_gradients]
densities = [den for _, den in density_gradients]

# Convert to numpy arrays for analysis
distances_np = np.array(distances)
densities_np = np.array(densities)

# Convert pixel distances to kilometers using satellite resolution
distances_km = distances_np * satellite_pixel_resolution / 1000  # Divide by 1000 to convert m to km

# Create a single figure with the density gradient plot
plt.figure(figsize=(8, 8))

# Plot the original density gradient
plt.plot(distances_km, densities_np, 'bo-', linewidth=2, label='Density Gradient')

# Find local centers (bumps/peaks)
if len(distances_np) > 5:
    prominence = 0.02 * (max(densities_np) - min(densities_np))
    peaks, _ = find_peaks(densities_np, prominence=prominence)
    
    # Find local minima (valleys)
    neg_densities = -1 * densities_np
    minima, _ = find_peaks(neg_densities, prominence=prominence)
else:
    peaks = []
    minima = []

# Skip the first point - don't add it to minima
# Add only last point if needed
if (len(distances_np)-1) not in peaks and (len(distances_np)-1) not in minima:
    minima = np.append(minima, np.array([len(distances_np)-1]))

# Sort minima to ensure they're in ascending order
minima = np.sort(minima)

# Calculate linear regression through minima
if len(minima) > 1:
    # Use scipy_stats to avoid conflicts with any existing 'stats' variable
    slope_pixel, intercept, r_value, p_value, std_err = scipy_stats.linregress(
        distances_np[minima], densities_np[minima])
    
    # Calculate slope in terms of km
    slope_km = slope_pixel * 1000 / satellite_pixel_resolution  # Convert to per-km
    
    # Create trendline for all points
    trendline = slope_pixel * distances_np + intercept
    
    # Define the y-value for which to calculate the intercept
    target_y_value = gradient_intercept  # Example value - adjust as needed
    
    # Calculate x-value where the line intersects target_y_value
    if abs(slope_pixel) > 1e-10:  # Avoid division by near-zero
        x_pixels_at_target_y = (target_y_value - intercept) / slope_pixel
        ld_distance_min_km = x_pixels_at_target_y * satellite_pixel_resolution / 1000  # Convert to km
    else:
        x_pixels_at_target_y = float('inf')  # If slope is essentially zero
        ld_distance_min_km = float('inf')
    
    # Plot trendline
    plt.plot(distances_km, trendline, 'g--', linewidth=2, 
             label=f'Alfa: {slope_km:.4f}/km\nDistance: {ld_distance_min_km:.2f} km')
    
    # Calculate MSE between actual and trendline
    mse = mean_squared_error(densities_np, trendline)
else:
    mse = 0
    ld_distance_min_km = float('inf')

# Add horizontal line at center threshold
plt.axhline(y=urban_center_threshold, color='r', linestyle='--', alpha=0.7, label=f"Centers Threshold ({urban_center_threshold})")

# Add target density to legend without drawing a line
plt.plot([], [], ' ', label=f"Target Density ({target_y_value:.2f})")

# If the intercept is within the plot range, mark it
if 0 <= ld_distance_min_km <= max(distances_km):
    plt.axvline(x=ld_distance_min_km, color='m', linestyle=':', alpha=0.5)
    plt.plot(ld_distance_min_km, target_y_value, 'mo', markersize=8)

# Set x-axis ticks using the defined interval
max_distance_km = np.ceil(max(distances_km))
x_ticks = np.arange(0, max_distance_km + 1, gradient_x_tick_interval/1000)  # Convert m to km
plt.xticks(x_ticks)
plt.grid(True, alpha=0.3)

plt.xlabel('Distance from Urban Center (km)')
plt.ylabel('Average Density')
plt.title(f'Density Gradient Through Minima (MSE: {mse:.6f})')
plt.legend()

# Make the plot have good proportions
plt.tight_layout()
plt.savefig(f'{output_dir}/{base_name}_urban_density_gradient.png', dpi=300)
plt.show()

In [ ]:
# Calculate density gradients from centers
def calculate_enhanced_density_gradients(centers_mask, urban_mask, density_map):
    """Calculate how density changes with distance from urban centers with enhanced statistics"""
    # Calculate distance from each urban pixel to the nearest center
    centers_dist_transform = distance_transform_edt(~centers_mask)
    
    # Only consider distances within urban areas
    urban_distances = centers_dist_transform[urban_mask]
    urban_densities = density_map[urban_mask]
    
    # Bin distances to create gradient profile with enhanced statistics
    max_dist = int(np.max(urban_distances)) + 1
    distance_bins = range(max_dist)
    density_stats_by_distance = []
    
    for d in distance_bins:
        # Get densities at this distance
        mask = (urban_distances >= d) & (urban_distances < d+1)
        if np.sum(mask) > 0:
            densities_at_d = urban_densities[mask]
            stats = {
                "distance": d,
                "min": np.min(densities_at_d),
                "q1": np.percentile(densities_at_d, 25),
                "median": np.median(densities_at_d),
                "mean": np.mean(densities_at_d),
                "q3": np.percentile(densities_at_d, 75),
                "max": np.max(densities_at_d)
            }
            density_stats_by_distance.append(stats)
    
    return density_stats_by_distance

# Calculate enhanced density gradients
density_stats = calculate_enhanced_density_gradients(urban_centers_mask, urban_mask, combined_image)

# Extract data for plotting - ensure all arrays are created from the same source
distances = np.array([stat["distance"] for stat in density_stats])
mean_densities = np.array([stat["mean"] for stat in density_stats])
median_densities = np.array([stat["median"] for stat in density_stats])
min_densities = np.array([stat["min"] for stat in density_stats])
max_densities = np.array([stat["max"] for stat in density_stats])
q1_densities = np.array([stat["q1"] for stat in density_stats])
q3_densities = np.array([stat["q3"] for stat in density_stats])

# Convert pixel distances to kilometers using satellite resolution
distances_km = distances * satellite_pixel_resolution / 1000  # Divide by 1000 to convert m to km

# Set gradient_intercept to the minimum of the Q1 values (minimum of IQR)
gradient_intercept = np.min(q1_densities)

# Find peaks and minima in the mean density profile
if len(distances) > 5:
    prominence = 0.02 * (np.max(mean_densities) - np.min(mean_densities))
    peaks, _ = find_peaks(mean_densities, prominence=prominence)
    
    # Find local minima (valleys)
    neg_densities = -1 * mean_densities
    minima, _ = find_peaks(neg_densities, prominence=prominence)
else:
    peaks = []
    minima = []

# Skip the first point - don't add it to minima
# Add only last point if needed
if (len(distances)-1) not in peaks and (len(distances)-1) not in minima:
    minima = np.append(minima, np.array([len(distances)-1]))

# Sort minima to ensure they're in ascending order
minima = np.sort(minima)

# Calculate linear regression through minima
if len(minima) > 1:
    slope_pixel, intercept, r_value, p_value, std_err = scipy_stats.linregress(
        distances[minima], mean_densities[minima])
    
    # Calculate slope in terms of km
    slope_km = slope_pixel * 1000 / satellite_pixel_resolution  # Convert to per-km
    
    # Create trendline for all points
    trendline = slope_pixel * distances + intercept
    
    # Calculate the residuals (mean density minus linear fitting)
    residuals = mean_densities - trendline
    
    # Calculate x-value where the line intersects target_y_value
    if abs(slope_pixel) > 1e-10:  # Avoid division by near-zero
        x_pixels_at_target_y = (gradient_intercept - intercept) / slope_pixel
        ld_distance_irq_km = x_pixels_at_target_y * satellite_pixel_resolution / 1000  # Convert to km
    else:
        x_pixels_at_target_y = float('inf')  # If slope is essentially zero
        ld_distance_irq_km = float('inf')
    
    # Calculate MSE between actual and trendline
    mse = mean_squared_error(mean_densities, trendline)
else:
    mse = 0
    ld_distance_irq_km = float('inf')
    slope_km = 0
    intercept = 0
    trendline = np.zeros_like(distances)
    residuals = np.zeros_like(distances)

# Create enhanced density gradient plot with two subplots (main plot and residuals)
fig = plt.figure(figsize=(10, 10))
# Create GridSpec to have more control over spacing
gs = fig.add_gridspec(2, 1, height_ratios=[3, 1])
ax1 = fig.add_subplot(gs[0])
ax2 = fig.add_subplot(gs[1], sharex=ax1)

# MAIN PLOT (TOP) - Now including both density data and fit in the same plot
# Plot density ranges on main plot
ax1.fill_between(distances_km, min_densities, max_densities, alpha=0.1, color='blue', label='Min-Max Range')
ax1.fill_between(distances_km, q1_densities, q3_densities, alpha=0.3, color='blue', label='IQR (25-75%)')

# Plot average and median lines on main plot
ax1.plot(distances_km, mean_densities, 'b-', linewidth=2, label='Mean Density')
ax1.plot(distances_km, median_densities, 'g-', linewidth=1.5, label='Median Density')

# Add the trendline on main plot
if len(minima) > 1:
    ax1.plot(distances_km, trendline, 'r--', linewidth=2, 
             label=f'Alfa: {slope_km:.4f}/km\nDistance: {ld_distance_irq_km:.2f} km')

# Add horizontal lines for thresholds on main plot
ax1.axhline(y=urban_center_threshold, color='r', linestyle='--', alpha=0.7, 
           label=f"Centers Threshold ({urban_center_threshold})")
ax1.axhline(y=urban_threshold, color='orange', linestyle='--', alpha=0.7, 
           label=f"Urban Threshold ({urban_threshold})")
ax1.axhline(y=gradient_intercept, color='m', linestyle='--', alpha=0.7, 
           label=f"IQR Min ({gradient_intercept:.2f})")

# Mark minima points used for regression on main plot
ax1.plot(distances_km[minima], mean_densities[minima], 'rx', markersize=10)

# If the intercept is within the plot range, mark it on main plot
if 0 <= ld_distance_irq_km <= max(distances_km):
    ax1.axvline(x=ld_distance_irq_km, color='m', linestyle=':', alpha=0.5)
    ax1.plot(ld_distance_irq_km, gradient_intercept, 'mo', markersize=8)

# Set title for main plot
ax1.set_title(f'Enhanced Density Gradient (MSE: {mse:.6f}, Intercept: {gradient_intercept:.4f})')
ax1.set_ylabel('Density')
ax1.grid(True, alpha=0.3)
ax1.legend(loc='upper right')

# RESIDUAL PLOT (BOTTOM)
if len(minima) > 1:
    # Create a filled area plot for residuals
    ax2.plot(distances_km, residuals, 'r-', linewidth=2, label='Density - Fit')
    ax2.fill_between(distances_km, 0, residuals, color='red', alpha=0.2)
    
    # Add horizontal line at y=0 for reference
    ax2.axhline(y=0, color='k', linestyle='-', alpha=0.5)
    
    # Set labels for residual plot
    ax2.set_ylabel('Density - Fit', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    # Add grid to residual plot
    ax2.grid(True, alpha=0.3)
    ax2.legend(loc='upper right')

# Set x-axis ticks using the defined interval (for both plots through sharex)
max_distance_km = np.ceil(max(distances_km))
x_ticks = np.arange(0, max_distance_km + 1, gradient_x_tick_interval/1000)  # Convert m to km
ax2.set_xticks(x_ticks)
ax2.set_xlabel('Distance from Urban Center (km)')

# Hide x-axis labels on top plot
plt.setp(ax1.get_xticklabels(), visible=False)

# Manually adjust the spacing instead of using tight_layout
plt.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9, hspace=0.0)

# Save and display the plot
plt.savefig(f'{output_dir}/{base_name}_enhanced_density_gradient.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
def segment_residuals_limited(distances_km, residuals, min_regions=2, max_regions=4):
    """
    Segment residuals into a limited number of regions (between min_regions and max_regions).
    
    Parameters:
    - distances_km: Array of distance values in km
    - residuals: Array of residual values (density - fit)
    - min_regions: Minimum number of regions to identify
    - max_regions: Maximum number of regions to identify
    
    Returns:
    - regions: List of tuples with (start_index, end_index, type) for identified regions
              where type is 'uniform' or 'variation'
    """
    import numpy as np
    from scipy.signal import find_peaks
    from sklearn.cluster import KMeans
    
    # Ensure we have enough data points
    if len(residuals) < min_regions:
        # If not enough data, return a single region
        return [(0, len(residuals)-1, 'unknown')]
    
    # Calculate metrics for segmentation
    abs_residuals = np.abs(residuals)
    gradients = np.gradient(residuals)
    abs_gradients = np.abs(gradients)
    
    # Prepare features for clustering
    # We use both residual values and their gradients for better segmentation
    features = np.column_stack((
        residuals,
        gradients,
        np.roll(gradients, 1),  # Shifted gradients to capture transitions
        np.roll(residuals, 1)   # Shifted residuals
    ))
    
    # Replace NaN values (from rolling)
    features[0, 2:] = features[1, 2:]
    
    # Try different numbers of clusters to find optimal segmentation
    best_regions = []
    best_score = -float('inf')
    
    for n_clusters in range(min_regions, max_regions + 1):
        # Apply KMeans clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(features)
        
        # Convert cluster labels to contiguous regions
        temp_regions = []
        current_label = labels[0]
        start_idx = 0
        
        for i in range(1, len(labels)):
            if labels[i] != current_label:
                # End of a region
                temp_regions.append((start_idx, i-1, current_label))
                current_label = labels[i]
                start_idx = i
        
        # Add the last region
        temp_regions.append((start_idx, len(labels)-1, current_label))
        
        # Merge small regions (less than 5% of total) with adjacent regions
        min_region_size = max(3, int(0.05 * len(residuals)))
        i = 0
        while i < len(temp_regions):
            start, end, label = temp_regions[i]
            if end - start + 1 < min_region_size:
                # This is a small region that should be merged
                if i > 0:
                    # Merge with previous region
                    prev_start, prev_end, prev_label = temp_regions[i-1]
                    temp_regions[i-1] = (prev_start, end, prev_label)
                    temp_regions.pop(i)
                elif i < len(temp_regions) - 1:
                    # Merge with next region
                    next_start, next_end, next_label = temp_regions[i+1]
                    temp_regions[i] = (start, next_end, next_label)
                    temp_regions.pop(i+1)
                else:
                    # This is the only region, keep it
                    i += 1
            else:
                i += 1
        
        # Calculate a score for this segmentation based on:
        # 1. Variance within each region (lower is better)
        # 2. Number of regions close to desired (closer to max_regions is better)
        within_variance = 0
        for start, end, label in temp_regions:
            region_residuals = residuals[start:end+1]
            within_variance += np.var(region_residuals) * (end - start + 1)
        
        within_variance /= len(residuals)
        
        # Penalize having too few regions
        region_count_score = -abs(len(temp_regions) - max_regions) if len(temp_regions) >= min_regions else -float('inf')
        
        # Total score (lower variance and more regions is better)
        score = region_count_score - within_variance
        
        if score > best_score:
            best_score = score
            best_regions = temp_regions
    
    # Classify each region as 'uniform' or 'variation'
    classified_regions = []
    
    for start, end, label in best_regions:
        region_residuals = residuals[start:end+1]
        region_std = np.std(region_residuals)
        
        # Use the overall standard deviation as a reference
        overall_std = np.std(residuals)
        
        if region_std < 0.7 * overall_std:
            region_type = 'uniform'
        else:
            region_type = 'variation'
        
        classified_regions.append((start, end, region_type))
    
    return classified_regions

def plot_segmented_residuals_limited(distances_km, residuals, regions):
    """
    Plot the residuals with segmentation highlighted using background colors
    and black dashed vertical lines for region boundaries.
    
    Parameters:
    - distances_km: Array of distance values in km
    - residuals: Array of residual values (density - fit)
    - regions: List of tuples with (start_index, end_index, type) for identified regions
    """
    import matplotlib.pyplot as plt
    import numpy as np
    from matplotlib.patches import Patch
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Define colors for region types with better contrast and transparency
    region_colors = {
        'uniform': 'lightgreen',
        'variation': 'lightsalmon',
        'unknown': 'lightgray'
    }
    
    # Ensure regions are ordered by start index
    sorted_regions = sorted(regions, key=lambda x: x[0])
    
    # First, add background colors for all regions and collect boundary points
    boundary_indices = []
    
    for i, (start, end, region_type) in enumerate(sorted_regions):
        # Add indices to boundary points
        if i == 0:  # First region
            boundary_indices.append(start)
        boundary_indices.append(end)
        
        # Color the background for this region
        color = region_colors.get(region_type)
        if i < len(sorted_regions) - 1:
            # For all regions except the last, go from start to the next region's start
            next_start = sorted_regions[i+1][0]
            x_start = distances_km[start]
            x_end = distances_km[next_start]
        else:
            # For the last region
            x_start = distances_km[start]
            x_end = distances_km[end]
        
        ax.axvspan(x_start, x_end, color=color, alpha=0.3, zorder=0)
    
    # Plot the residuals line on top of the background
    ax.plot(distances_km, residuals, 'k-', linewidth=1.5, label='Residuals (Density - Fit)', zorder=3)
    
    # Fill the area under the residuals curve with semi-transparent gray
    ax.fill_between(distances_km, 0, residuals, color='gray', alpha=0.2, zorder=2)
    
    # Add a horizontal line at zero
    ax.axhline(y=0, color='black', linestyle='-', alpha=0.5, zorder=1)
    
    # Get y limits for vertical lines
    y_min, y_max = ax.get_ylim()
    
    # Mark only unique boundary points on the x-axis with vertical dashed lines
    for i, idx in enumerate(boundary_indices):
        x = distances_km[idx]
        
        # Add vertical dashed line from bottom to top
        ax.axvline(x=x, color='black', linestyle='--', linewidth=1, alpha=0.7, zorder=1)
        
        # Mark boundary on x-axis with a triangle
        ax.plot(x, 0, 'o', color='red', markersize=8, markeredgewidth=1, 
               markeredgecolor='white', zorder=4)
        
        # Add distance label at boundary
        ax.text(x, 0, f' {x:.2f}', verticalalignment='bottom', 
               horizontalalignment='left', fontsize=10, rotation=45, color='red')
    
    # Add region labels in the middle of each region
    for i, (start, end, region_type) in enumerate(sorted_regions):
        # For label position, use the midpoint between this region's start and the next region's start
        if i < len(sorted_regions) - 1:
            next_start = sorted_regions[i+1][0]
            region_mid = (distances_km[start] + distances_km[next_start]) / 2
        else:
            region_mid = (distances_km[start] + distances_km[end]) / 2
            
        if region_type == 'uniform':
            text_color = 'darkgreen'
        else:
            text_color = 'darkred'
        ax.text(region_mid, 0, f'R{i+1}', verticalalignment='bottom', 
               horizontalalignment='center', fontsize=10, fontweight='bold', 
               color=text_color, bbox=dict(facecolor='white', alpha=0.7, pad=2))
    
    # Create a custom legend with region information
    legend_elements = [
        Patch(facecolor=region_colors['uniform'], alpha=0.3, edgecolor='green', 
              label='Uniform Regions'),
        Patch(facecolor=region_colors['variation'], alpha=0.3, edgecolor='red', 
              label='High-Variation Regions')
    ]
    
    # Add the residuals line to the legend elements
    legend_elements.append(plt.Line2D([0], [0], color='black', linewidth=1.5, 
                                    label='Residuals (Density - Fit)'))
    
    # Create a separate text box with region details
    region_info = []
    for i, (start, end, region_type) in enumerate(sorted_regions):
        # For reporting region extents, use the next region's start as the end boundary
        if i < len(sorted_regions) - 1:
            next_start = sorted_regions[i+1][0]
            x_start = distances_km[start]
            x_end = distances_km[next_start]
        else:
            x_start = distances_km[start]
            x_end = distances_km[end]
            
        if region_type == 'uniform':
            color_name = 'green'
        else:
            color_name = 'red'
        region_info.append(f"Region {i+1} ({x_start:.2f} - {x_end:.2f} km): {region_type.capitalize()}")
    
    # Add the region information text box
    props = dict(boxstyle='round', facecolor='white', alpha=0.8)
    ax.text(0.02, 0.98, '\n'.join(region_info), transform=ax.transAxes, fontsize=9,
           verticalalignment='top', bbox=props)
    
    # Add labels and title
    ax.set_xlabel('Distance from Urban Center (km)')
    ax.set_ylabel('Density - Fit')
    ax.set_title('Segmentation of Residuals')
    ax.grid(True, alpha=0.3, zorder=0)
    
    # Add the main legend
    ax.legend(handles=legend_elements, loc='lower right')
    
    plt.tight_layout()
    return fig

def segment_residuals_limited(distances_km, residuals, min_regions=2, max_regions=4):
    """
    Segment residuals into a limited number of regions (between min_regions and max_regions).
    Ensures that region boundaries match exactly.
    
    Parameters:
    - distances_km: Array of distance values in km
    - residuals: Array of residual values (density - fit)
    - min_regions: Minimum number of regions to identify
    - max_regions: Maximum number of regions to identify
    
    Returns:
    - regions: List of tuples with (start_index, end_index, type) for identified regions
              where type is 'uniform' or 'variation'
    """
    import numpy as np
    from scipy.signal import find_peaks
    from sklearn.cluster import KMeans
    
    # Ensure we have enough data points
    if len(residuals) < min_regions:
        # If not enough data, return a single region
        return [(0, len(residuals)-1, 'unknown')]
    
    # Calculate metrics for segmentation
    abs_residuals = np.abs(residuals)
    gradients = np.gradient(residuals)
    abs_gradients = np.abs(gradients)
    
    # Prepare features for clustering
    # We use both residual values and their gradients for better segmentation
    features = np.column_stack((
        residuals,
        gradients,
        np.roll(gradients, 1),  # Shifted gradients to capture transitions
        np.roll(residuals, 1)   # Shifted residuals
    ))
    
    # Replace NaN values (from rolling)
    features[0, 2:] = features[1, 2:]
    
    # Try different numbers of clusters to find optimal segmentation
    best_regions = []
    best_score = -float('inf')
    
    for n_clusters in range(min_regions, max_regions + 1):
        # Apply KMeans clustering
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(features)
        
        # Convert cluster labels to contiguous regions
        temp_regions = []
        current_label = labels[0]
        start_idx = 0
        
        for i in range(1, len(labels)):
            if labels[i] != current_label:
                # End of a region
                temp_regions.append((start_idx, i-1, current_label))
                current_label = labels[i]
                start_idx = i
        
        # Add the last region
        temp_regions.append((start_idx, len(labels)-1, current_label))
        
        # Merge small regions (less than 5% of total) with adjacent regions
        min_region_size = max(3, int(0.05 * len(residuals)))
        i = 0
        while i < len(temp_regions):
            start, end, label = temp_regions[i]
            if end - start + 1 < min_region_size:
                # This is a small region that should be merged
                if i > 0:
                    # Merge with previous region
                    prev_start, prev_end, prev_label = temp_regions[i-1]
                    temp_regions[i-1] = (prev_start, end, prev_label)
                    temp_regions.pop(i)
                elif i < len(temp_regions) - 1:
                    # Merge with next region
                    next_start, next_end, next_label = temp_regions[i+1]
                    temp_regions[i] = (start, next_end, next_label)
                    temp_regions.pop(i+1)
                else:
                    # This is the only region, keep it
                    i += 1
            else:
                i += 1
        
        # Calculate a score for this segmentation based on:
        # 1. Variance within each region (lower is better)
        # 2. Number of regions close to desired (closer to max_regions is better)
        within_variance = 0
        for start, end, label in temp_regions:
            region_residuals = residuals[start:end+1]
            within_variance += np.var(region_residuals) * (end - start + 1)
        
        within_variance /= len(residuals)
        
        # Penalize having too few regions
        region_count_score = -abs(len(temp_regions) - max_regions) if len(temp_regions) >= min_regions else -float('inf')
        
        # Total score (lower variance and more regions is better)
        score = region_count_score - within_variance
        
        if score > best_score:
            best_score = score
            best_regions = temp_regions
    
    # Sort regions by start index to ensure they're in order
    best_regions.sort(key=lambda x: x[0])
    
    # Ensure the first region starts at index 0
    if best_regions[0][0] > 0:
        best_regions[0] = (0, best_regions[0][1], best_regions[0][2])
    
    # Ensure the last region ends at the last data point
    if best_regions[-1][1] < len(residuals) - 1:
        best_regions[-1] = (best_regions[-1][0], len(residuals) - 1, best_regions[-1][2])
    
    # Create perfectly contiguous regions
    contiguous_regions = []
    for i, (start, end, label) in enumerate(best_regions):
        if i == 0:
            # First region starts at 0
            contiguous_regions.append((0, end, label))
        else:
            # Each subsequent region starts exactly where the previous one ended
            prev_end = contiguous_regions[-1][1]
            contiguous_regions.append((prev_end + 1, end, label))
    
    # Make sure there are no gaps between regions
    for i in range(1, len(contiguous_regions)):
        prev_end = contiguous_regions[i-1][1]
        current_start = contiguous_regions[i][0]
        
        if current_start != prev_end + 1:
            # Fix the gap
            contiguous_regions[i] = (prev_end + 1, contiguous_regions[i][1], contiguous_regions[i][2])
    
    # Classify each region as 'uniform' or 'variation'
    classified_regions = []
    
    for start, end, label in contiguous_regions:
        region_residuals = residuals[start:end+1]
        region_std = np.std(region_residuals)
        
        # Use the overall standard deviation as a reference
        overall_std = np.std(residuals)
        
        if region_std < 0.7 * overall_std:
            region_type = 'uniform'
        else:
            region_type = 'variation'
        
        classified_regions.append((start, end, region_type))
    
    return classified_regions

def analyze_segmented_residuals_limited(distances_km, residuals, regions):
    """
    Analyze the segmented residuals and print detailed statistics for the limited regions.
    
    Parameters:
    - distances_km: Array of distance values in km
    - residuals: Array of residual values (density - fit)
    - regions: List of tuples with (start_index, end_index, type) for identified regions
    """
    import numpy as np
    from scipy import stats
    
    print("=== RESIDUALS SEGMENTATION ANALYSIS (LIMITED REGIONS) ===")
    
    # Overall stats
    print("\nOVERALL STATISTICS:")
    print(f"Total distance analyzed: {distances_km[-1] - distances_km[0]:.2f} km")
    print(f"Number of data points: {len(residuals)}")
    print(f"Mean residual: {np.mean(residuals):.4f}")
    print(f"Standard deviation: {np.std(residuals):.4f}")
    print(f"Range: {np.max(residuals) - np.min(residuals):.4f}")
    
    # Region analysis
    print(f"\nIDENTIFIED REGIONS: {len(regions)}")
    
    for i, (start, end, region_type) in enumerate(regions):
        region_distances = distances_km[start:end+1]
        region_residuals = residuals[start:end+1]
        region_length = region_distances[-1] - region_distances[0]
        
        print(f"\nRegion {i+1} ({region_type.upper()}):")
        print(f"  Distance range: {region_distances[0]:.2f} km to {region_distances[-1]:.2f} km (Length: {region_length:.2f} km)")
        print(f"  Mean: {np.mean(region_residuals):.4f}")
        print(f"  Standard deviation: {np.std(region_residuals):.4f}")
        print(f"  Range: {np.max(region_residuals) - np.min(region_residuals):.4f}")
        
        if region_type == 'variation':
            # Calculate peaks
            from scipy.signal import find_peaks
            peaks, _ = find_peaks(region_residuals, prominence=0.01)
            valleys, _ = find_peaks(-region_residuals, prominence=0.01)
            
            if len(peaks) > 0:
                peak_distances = [region_distances[p] for p in peaks]
                peak_values = [region_residuals[p] for p in peaks]
                print(f"  Peaks: {len(peaks)} found at distances: {', '.join([f'{d:.2f} km' for d in peak_distances])}")
                print(f"  Peak values: {', '.join([f'{v:.4f}' for v in peak_values])}")
            
            if len(valleys) > 0:
                valley_distances = [region_distances[v] for v in valleys]
                valley_values = [region_residuals[v] for v in valleys]
                print(f"  Valleys: {len(valleys)} found at distances: {', '.join([f'{d:.2f} km' for d in valley_distances])}")
                print(f"  Valley values: {', '.join([f'{v:.4f}' for v in valley_values])}")
    
    print("\n=== END OF ANALYSIS ===")

# Using the new limited segmentation approach
# Segment the residuals with limited regions
regions = segment_residuals_limited(
    distances_km, 
    residuals, 
    min_regions=1,    # Minimum number of regions to identify
    max_regions=3     # Maximum number of regions to identify
)

# Plot the segmented residuals
fig = plot_segmented_residuals_limited(distances_km, residuals, regions)

# Analyze and print detailed statistics
analyze_segmented_residuals_limited(distances_km, residuals, regions)

# Save the segmentation plot
output_segmentation_path = f'{output_dir}/{base_name}_residual_segmentation_limited.png'
fig.savefig(output_segmentation_path, dpi=300, bbox_inches='tight')
print(f"Segmentation plot saved to: {output_segmentation_path}")

In [ ]:
# Define distance for circle radius
ld_distance_km = ld_distance_irq_km / 2

# Define center sizse selection for plots
center_min_size = 000
center_max_size = 5

# Find centers of urban center clusters
num_centers, center_labels, center_stats, center_centroids = cv2.connectedComponentsWithStats(
    urban_centers_mask.astype(np.uint8), connectivity=8)

# Convert combined image to a grayscale background
# Normalize to 0-255 range
background_image = np.clip(combined_image * 255 / combined_image.max(), 0, 255).astype(np.uint8)
# Convert to 3-channel grayscale image for colored overlays
background_image_rgb = cv2.cvtColor(background_image, cv2.COLOR_GRAY2RGB)

# Create a copy for drawing
result_image = background_image_rgb.copy()

# Create a separate mask for all circles
circles_mask = np.zeros_like(result_image)
radius_pixels = int(ld_distance_km * 1000 / satellite_pixel_resolution)  # Convert km to pixels

# Define red color in RGB format for consistency with matplotlib
red_color = (255, 0, 0)  # This is RED in RGB format

# Draw all circles on the circles mask
for i in range(1, num_centers):
    # Skip clusters not in the desired size range
    if (center_stats[i, cv2.CC_STAT_AREA] < center_min_size) or (center_stats[i, cv2.CC_STAT_AREA] > center_max_size):
        continue
        
    # Get center coordinates and convert to integers
    center_y, center_x = center_centroids[i]
    center = (int(center_x), int(center_y))
    
    # Draw a filled circle on the circles mask (solid red)
    cv2.circle(circles_mask, center, radius_pixels, red_color, -1)  # Red filled circle
    
    # Draw circle boundary in red (same color as fill)
    cv2.circle(circles_mask, center, radius_pixels, red_color, 2)  # Red border

    break

# Now combine the background and circles with proper alpha
# Create binary mask where circles exist
binary_mask = (circles_mask > 0).any(axis=2)
# Apply the circles with alpha=0.2 only where circles exist
alpha = 0.2
result_image[binary_mask] = cv2.addWeighted(
    circles_mask[binary_mask], 
    alpha, 
    result_image[binary_mask], 
    1 - alpha, 
    0
)

# Convert to BGR for saving with cv2.imwrite
result_image_bgr = cv2.cvtColor(result_image, cv2.COLOR_RGB2BGR)
cv2.imwrite(f'data/{base_name}_urban_centers_circles_gray.png', result_image_bgr)

# Display using matplotlib (which expects RGB)
plt.figure(figsize=(10, 8))
plt.imshow(result_image)
set_axes_in_meters(plt.gca(), result_image.shape)
plt.title(f'Urban Centers with {ld_distance_km:.2f} km Radius Circles')
plt.tight_layout()
plt.show()